In [111]:
import json
import glob
import os.path
import pandas as pd
import numpy as np


In [112]:
def gettblData(tbl):# hitter 정보 테이블
    li = []
    for row in tbl['rows']:
        subli = []
        for r in row['row']:
            subli.append(r['Text'])
        li.append(subli)
    return pd.DataFrame(li)

# 타자 기록 2팀.
def get_hitter_record(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())

    result = []
    for hitter in boxscoreData['arrHitter']:
        df_hitter_record = pd.DataFrame()
        # tbl1 # 이닝/포지션/이름
        # tbl2 # 이닝별 성적
        # tbl3 # 타수 안타 타점 득점 타율
        # hitter = boxscoreData['arrHitter'][1] # table1 ~ 3
        for key in hitter: # 3구역으로 구성된 hitter 테이블을 하나씩 pick
            tbl = json.loads(hitter[key])
            df_hitter_record = pd.concat([df_hitter_record, gettblData(tbl)], axis=1)

        # 컬럼 이름 지정
        idx = ['이닝','포지션','선수명']
        idx.extend([str(i+1) for i in list(range(boxscoreData['realMaxInning']))])
        idx.extend(['타수','안타','타점','득점','타율'])
        df_hitter_record.columns = idx
        result.append(df_hitter_record)
    return result

def get_Pitcher_record(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())
    result = []

    for pitcher in boxscoreData['arrPitcher']:
        tbl = json.loads(pitcher['table'])
        pitcher_df = gettblData(tbl)
        pitcher_df.columns = ['선수명','등판','결과','승','패','세','이닝','타자','투구수','타수','피안타','홈런','4사구','삼진','실점','자책','평균자책점']
        result.append(pitcher_df)
    return result

def get_Summary(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())
    data = json.loads(boxscoreData['tableEtc'])
    return gettblData(data)

def getScoreBoard(path):
    data = json.loads(open(path, 'r', encoding='utf-8-sig').read())

    tbl1_data = json.loads(data['table1'])
    tbl2_data = json.loads(data['table2'])
    tbl3_data = json.loads(data['table3'])

    scoreboardDF = {}
    scoreboardDF['away_승패'] = tbl1_data['rows'][0]['row'][0]['Text']
    scoreboardDF['away_전적'] = tbl1_data['rows'][0]['row'][1]['Text'].split('</span>')[1]
    inning = 1
    for i in tbl2_data['rows'][0]['row']:
        scoreboardDF['away_' + str(inning)] = i['Text']
        inning += 1
    scoreboardDF['away_R'] = tbl3_data['rows'][0]['row'][0]['Text'] # R 점수
    scoreboardDF['away_H'] = tbl3_data['rows'][0]['row'][1]['Text'] # H 안타
    scoreboardDF['away_E'] = tbl3_data['rows'][0]['row'][2]['Text'] # E 에러
    scoreboardDF['away_B'] = tbl3_data['rows'][0]['row'][3]['Text'] # 볼넷 수

    scoreboardDF['home_승패'] = tbl1_data['rows'][1]['row'][0]['Text']
    scoreboardDF['home_전적'] = tbl1_data['rows'][1]['row'][1]['Text'].split('</span>')[1]
    inning = 1
    for i in tbl2_data['rows'][1]['row']:
        scoreboardDF['home_' + str(inning)] = i['Text']
        inning += 1
    scoreboardDF['home_R'] = tbl3_data['rows'][1]['row'][0]['Text'] # R 점수
    scoreboardDF['home_H'] = tbl3_data['rows'][1]['row'][1]['Text'] # H 안타
    scoreboardDF['home_E'] = tbl3_data['rows'][1]['row'][2]['Text'] # E 에러
    scoreboardDF['home_B'] = tbl3_data['rows'][1]['row'][3]['Text'] # 볼넷 수

    return pd.concat([pd.DataFrame(data, index=[0]), pd.DataFrame(scoreboardDF, index=[0])], axis=1)

In [15]:
# SR_ID
# 포스트 시즌 : 4
# 시범경기 : 1
# 정규시즌 : 0


dirpath = r'.\teamReviewRawData_json'

## test용 파일 경로
filename1 = r'0_2021_20210403SSWO0_GetBoxScoreScroll.json'
filename2 = r'0_2021_20210403SSWO0_GetScoreBoardScroll.json'
boxscorePath = os.path.join(dirpath, filename1)
scoreboardPath = os.path.join(dirpath, filename2)

# get_Gameinfo(scoreboardPath)
# getScoreboardDataFrame(scoreboardPath) # 스코어보드

# get_Summary(boxscorePath) # 요약 정보
# get_hitter_record(boxscorePath)[0] # 타자 기록 : 원정팀
# get_hitter_record(boxscorePath)[1] # 타자 기록 : 홈팀
# get_Pitcher_record(boxscorePath)[0] # 투수 기록 : 원정팀
# get_Pitcher_record(boxscorePath)[1] # 투수 기록 : 홈팀

,0
SR_ID,0
G_ID,20210403SSWO0
G_DT,2021-04-03
SEASON_ID,2021
HOME_NM,키움
HOME_ID,WO
AWAY_NM,삼성
AWAY_ID,SS
S_NM,고척
CROWD_CN,"1,665"


In [97]:
result = pd.DataFrame()

for filename in os.listdir(dirpath):
    if filename.__contains__('GetScoreBoardScroll') == False:
        continue
    path = os.path.join(dirpath,filename)
    result = result.append(getScoreBoard(path))
    print(f'>>> {path} is Load Complete!!!')


result.reset_index(drop=True, inplace=True)
# result.to_csv('playReview_ScoreBoard_Rawdata.csv', encoding='utf-8-sig', index=False)

>>> .\teamReviewRawData_json\0_2018_20180324HHWO0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180324KTHT0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180324LGNC0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180324LTSK0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180324SSOB0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180325HHWO0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180325KTHT0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180325LGNC0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180325LTSK0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180325SSOB0_GetScoreBoardScroll.json is Load Complete!!!
>>> .\teamReviewRawData_json\0_2018_20180327HHNC0_

In [113]:
result

,LE_ID,SR_ID,G_ID,G_DT,SEASON_ID,HOME_NM,HOME_ID,AWAY_NM,AWAY_ID,S_NM,...,home_7,home_8,home_9,home_10,home_11,home_12,home_R,home_H,home_E,home_B
0,1,0,20180324HHWO0,2018-03-24,2018,넥센,WO,한화,HH,고척,...,0,0,-,-,-,-,6,12,1,4
1,1,0,20180324KTHT0,2018-03-24,2018,KIA,HT,KT,KT,광주,...,0,0,0,-,-,-,4,12,0,4
2,1,0,20180324LGNC0,2018-03-24,2018,NC,NC,LG,LG,마산,...,0,2,-,-,-,-,4,9,0,4
3,1,0,20180324LTSK0,2018-03-24,2018,SK,SK,롯데,LT,문학,...,1,0,-,-,-,-,6,8,3,7
4,1,0,20180324SSOB0,2018-03-24,2018,두산,OB,삼성,SS,잠실,...,2,0,0,-,-,-,3,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,1,0,20211030LGLT0,2021-10-30,2021,롯데,LT,LG,LG,사직,...,0,0,-,-,-,-,4,9,0,4
2877,1,0,20211030OBHH0,2021-10-30,2021,한화,HH,두산,OB,대전,...,0,0,0,-,-,-,3,7,2,1
2878,1,0,20211030SSNC0,2021-10-30,2021,NC,NC,삼성,SS,창원,...,0,0,1,-,-,-,5,10,1,4
2879,1,0,20211030WOHT0,2021-10-30,2021,KIA,HT,키움,WO,광주,...,0,0,0,-,-,-,1,5,0,0
